In [19]:
import os
import cv2
import numpy as np
import pandas as pd
import h5py
import glob

In [16]:
def events_to_voxel_grid(event_file, events, num_bins, width, height):
    """
    Build a voxel grid with bilinear interpolation in the time domain from a set of events.

    events : [timestamp, x, y, polarity]
    num_bins : number of bins in the temporal axis of the voxel grid
    """
    try:
        assert events.shape[1] == 4
    except IndexError:
        raise ValueError(f"{event_file} must have 4 columns...")
    
    assert(num_bins > 0)
    assert(width > 0)
    assert(height > 0)

    voxel_grid = np.zeros((num_bins, height, width), np.float32).ravel()

    # normalize the event timestamps so that they lie between 0 and num_bins
    last_stamp = events[-1, 0]
    first_stamp = events[0, 0]
    deltaT = last_stamp - first_stamp

    if deltaT == 0:
        deltaT = 1.0

    events[:, 0] = (num_bins - 1) * (events[:, 0] - first_stamp) / deltaT
    ts = events[:, 0]
    xs = events[:, 1].astype(np.int32)
    ys = events[:, 2].astype(np.int32)
    pols = events[:, 3]
    pols[pols == 0] = -1  # polarity should be +1 / -1

    tis = ts.astype(np.int32)
    dts = ts - tis
    vals_left = pols * (1.0 - dts)
    vals_right = pols * dts

    valid_indices = tis < num_bins
    np.add.at(voxel_grid, xs[valid_indices] + ys[valid_indices] * width
              + tis[valid_indices] * width * height, vals_left[valid_indices])

    valid_indices = (tis + 1) < num_bins
    np.add.at(voxel_grid, xs[valid_indices] + ys[valid_indices] * width
              + (tis[valid_indices] + 1) * width * height, vals_right[valid_indices])

    voxel_grid = np.reshape(voxel_grid, (num_bins, height, width))

    return voxel_grid

In [17]:
def convert_npy_to_hdf(npy_event_dir, image_shape=(625, 970), num_bins=10):
    # 모든 시퀀스 폴더 찾기
    sequence_dirs = [d for d in os.listdir(npy_event_dir) if os.path.isdir(os.path.join(npy_event_dir, d))]

    for sequence_dir in sequence_dirs:
        events_dir = os.path.join(npy_event_dir, sequence_dir, "events_npy")
        if not os.path.exists(events_dir):
            continue
            
        npy_files = sorted(glob(os.path.join(events_dir, "*.npy")))
        hdf_event_dir = os.path.join(npy_event_dir, sequence_dir, "events_hdf")
        os.makedirs(hdf_event_dir, exist_ok=True)

        for npy_path in npy_files:
            try:
                event_npy = np.load(npy_path)
                voxel_grid = events_to_voxel_grid(npy_path, event_npy, num_bins, image_shape[1], image_shape[0])

                fname = os.path.basename(npy_path).replace(".npy", ".hdf")
                save_path = os.path.join(hdf_event_dir, fname)

                with h5py.File(save_path, 'w') as f:
                    f.create_dataset('data', data=voxel_grid, dtype=np.float32)

                print(f"✅ Saved: {save_path}")
            except Exception as e:
                print(f"❌ Failed: {npy_path} — {e}")

In [22]:
def convert_npy_to_hdf(npy_event_dir, num_bins=10):
    # 모든 시퀀스 폴더 찾기
    sequence_dirs = [d for d in os.listdir(npy_event_dir) if os.path.isdir(os.path.join(npy_event_dir, d))]

    for sequence_dir in sequence_dirs:
        sequence_path = os.path.join(npy_event_dir, sequence_dir)
        events_dir = os.path.join(sequence_path, "events_npy")
        images_dir = os.path.join(sequence_path, "images_corrected")

        if not os.path.exists(events_dir):
            continue
        if not os.path.exists(images_dir):
            continue


        # 이미지 해상도 얻기
        sample_image_path = sorted(glob.glob(os.path.join(images_dir, "*.png")))[0]
        sample_image = cv2.imread(sample_image_path, cv2.IMREAD_GRAYSCALE)
        height, width = sample_image.shape
            
        npy_files = sorted(glob.glob(os.path.join(events_dir, "*.npy")))
        hdf_event_dir = os.path.join(npy_event_dir, sequence_dir, "events_hdf")
        os.makedirs(hdf_event_dir, exist_ok=True)

        for npy_path in npy_files:
            try:
                event_npy = np.load(npy_path)
                voxel_grid = events_to_voxel_grid(npy_path, event_npy, num_bins, width, height)

                fname = os.path.basename(npy_path).replace(".npy", ".hdf")
                save_path = os.path.join(hdf_event_dir, fname)

                with h5py.File(save_path, 'w') as f:
                    f.create_dataset('data', data=voxel_grid, dtype=np.float32)

                print(f"✅ Saved: {save_path}")
            except Exception as e:
                print(f"❌ Failed: {npy_path} — {e}")

In [23]:
path = "/scratch2/jiyun.kong/hs_ergb/far"
convert_npy_to_hdf(path)

✅ Saved: /scratch2/jiyun.kong/hs_ergb/far/lake_03/events_hdf/000000.hdf
✅ Saved: /scratch2/jiyun.kong/hs_ergb/far/lake_03/events_hdf/000001.hdf
✅ Saved: /scratch2/jiyun.kong/hs_ergb/far/lake_03/events_hdf/000002.hdf
✅ Saved: /scratch2/jiyun.kong/hs_ergb/far/lake_03/events_hdf/000003.hdf
✅ Saved: /scratch2/jiyun.kong/hs_ergb/far/lake_03/events_hdf/000004.hdf
✅ Saved: /scratch2/jiyun.kong/hs_ergb/far/lake_03/events_hdf/000005.hdf
✅ Saved: /scratch2/jiyun.kong/hs_ergb/far/lake_03/events_hdf/000006.hdf
✅ Saved: /scratch2/jiyun.kong/hs_ergb/far/lake_03/events_hdf/000007.hdf
✅ Saved: /scratch2/jiyun.kong/hs_ergb/far/lake_03/events_hdf/000008.hdf
✅ Saved: /scratch2/jiyun.kong/hs_ergb/far/lake_03/events_hdf/000009.hdf
✅ Saved: /scratch2/jiyun.kong/hs_ergb/far/lake_03/events_hdf/000010.hdf
✅ Saved: /scratch2/jiyun.kong/hs_ergb/far/lake_03/events_hdf/000011.hdf
✅ Saved: /scratch2/jiyun.kong/hs_ergb/far/lake_03/events_hdf/000012.hdf
✅ Saved: /scratch2/jiyun.kong/hs_ergb/far/lake_03/events_hdf/000

In [24]:
path = "/scratch2/jiyun.kong/hs_ergb/close"
convert_npy_to_hdf(path)

❌ Failed: /scratch2/jiyun.kong/hs_ergb/close/spinning_plate/events_npy/000000.npy — index -1 is out of bounds for axis 0 with size 0
✅ Saved: /scratch2/jiyun.kong/hs_ergb/close/spinning_plate/events_hdf/000001.hdf
✅ Saved: /scratch2/jiyun.kong/hs_ergb/close/spinning_plate/events_hdf/000002.hdf
✅ Saved: /scratch2/jiyun.kong/hs_ergb/close/spinning_plate/events_hdf/000003.hdf
✅ Saved: /scratch2/jiyun.kong/hs_ergb/close/spinning_plate/events_hdf/000004.hdf
✅ Saved: /scratch2/jiyun.kong/hs_ergb/close/spinning_plate/events_hdf/000005.hdf
✅ Saved: /scratch2/jiyun.kong/hs_ergb/close/spinning_plate/events_hdf/000006.hdf
✅ Saved: /scratch2/jiyun.kong/hs_ergb/close/spinning_plate/events_hdf/000007.hdf
✅ Saved: /scratch2/jiyun.kong/hs_ergb/close/spinning_plate/events_hdf/000008.hdf
✅ Saved: /scratch2/jiyun.kong/hs_ergb/close/spinning_plate/events_hdf/000009.hdf
✅ Saved: /scratch2/jiyun.kong/hs_ergb/close/spinning_plate/events_hdf/000010.hdf
✅ Saved: /scratch2/jiyun.kong/hs_ergb/close/spinning_plat

In [ ]:
path = "/scratch2/jiyun.kong/bs_ergb/1_TEST"
convert_npy_to_hdf(path)


In [ ]:
path = "/scratch2/jiyun.kong/bs_ergb/2_VALIDATION"
convert_npy_to_hdf(path)

In [ ]:
path = "/scratch2/jiyun.kong/bs_ergb/3_TRAINING"
convert_npy_to_hdf(path)

### FPS 계산하기

In [30]:
# 시퀀스 경로
sequence_base_path = "/scratch2/jiyun.kong/hs_ergb/far/"
sequence_list = os.listdir(sequence_base_path)

fps_results = []

for seq_name in sequence_list:
    try:
        timestamp_path = os.path.join(sequence_base_path, seq_name, "images_corrected", "timestamp.txt")
        image_path_pattern = os.path.join(sequence_base_path, seq_name, "images_corrected", "*.png")
        image_list = sorted(glob.glob(image_path_pattern))

        if not os.path.exists(timestamp_path):
            continue

        # 타임스탬프 불러오기
        with open(timestamp_path, "r") as f:
            timestamps = f.readlines()
        timestamps = [float(x.strip()) for x in timestamps]

        # 프레임 수
        num_frames = len(image_list)

        # FPS 계산
        total_time_us = timestamps[-1] - timestamps[0]  # 마이크로초 단위
        total_time_s = total_time_us / 1e6
        estimated_fps = (len(timestamps)) / total_time_s  # 프레임 수 / 총 시간

        fps_results.append((seq_name, round(estimated_fps, 2), num_frames))
    except Exception as e:
        fps_results.append((seq_name, "Error", str(e)))

In [31]:
fps_results

[('lake_03', 162.87, 833),
 ('bridge_lake_03', 162.88, 834),
 ('kornhausbruecke_letten_random_04', 162.87, 834),
 ('bridge_lake_01', 162.87, 833),
 ('lake_01', 162.86, 784),
 ('sihl_03', 162.86, 834)]

In [32]:
df = pd.DataFrame(fps_results, columns=["Sequence", "Estimated FPS", "# of PNG Images"])

# CSV로 저장
csv_path = "/home/jiyun.kong/VFPSIE/data/far_summary.csv"
df.to_csv(csv_path, index=False)

csv_path

'/home/jiyun.kong/VFPSIE/data/far_summary.csv'